In [70]:
###Libraries and imports
import numpy as np
import math
import random
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Conv3D, Conv3DTranspose, MaxPooling3D, BatchNormalization, Dropout, Activation, Input, concatenate
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
from scipy.ndimage import zoom

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
#Set Root Directory
ROOT_DIR = '/content/drive/My Drive/BraTS2021'

In [73]:
# Extract file IDs dynamically from the available files in the ROOT_DIR
all_files = os.listdir(ROOT_DIR)

# Extract unique file IDs (e.g., '00495') based on file naming conventions
file_ids = sorted(list(set(f.split('_')[1] for f in all_files if f.startswith("BraTS2021") and f.endswith(".nii.gz"))))

In [74]:
#Parameters
PARAMS = {
    'orig_dim': (240, 240, 155),  # Original image dimensions
    'final_dim': (64, 64, 64),   # Input/output size for the model
    'batch_size': 1,
    'n_classes': 4,
    'n_channels': 4,
    'channel_keys': ['t2', 't1ce', 't1', 'flair'],
    'label_key': 'seg',
    'shuffle': True
}

In [75]:
# Utility function for resizing
def resize_3d(image, target_shape):
    zoom_factors = [t / s for t, s in zip(target_shape, image.shape)]
    return zoom(image, zoom_factors, order=1)

In [82]:
# Data Generator
class DataGenerator(Sequence):
    def __init__(self, file_ids, **kwargs):
        self.file_ids = file_ids
        self.params = kwargs
        self.shuffle = kwargs.get('shuffle', True)
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.file_ids) / self.params['batch_size']))

    def __getitem__(self, index):
        batch_ids = self.file_ids[index * self.params['batch_size']:(index + 1) * self.params['batch_size']]
        X, y = self.__data_generation(batch_ids)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.file_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_ids):
        X = np.empty((self.params['batch_size'], *self.params['final_dim'], self.params['n_channels']))
        y = np.empty((self.params['batch_size'], *self.params['final_dim']), dtype=int)

        for i, file_id in enumerate(batch_ids):
            X[i,], y[i,] = self.load_sample(file_id)

        return X, tf.keras.utils.to_categorical(y, num_classes=self.params['n_classes'])

    def load_sample(self, file_id):
    # Load each MRI modality
      image_data = [self.load_nifti(file_id, key) for key in self.params['channel_keys']]
      mask_data = self.load_nifti(file_id, self.params['label_key'])

    # Resize the images and mask
      image_data = [resize_3d(img, self.params['final_dim']) for img in image_data]
      mask_data = resize_3d(mask_data, self.params['final_dim'])

    # Clean up and remap mask values
      mask_data = np.round(mask_data)  # Round values to nearest integer
      mask_data = mask_data.astype(int)  # Convert to integer type
      mask_data = np.clip(mask_data, 0, 3)  # Ensure all values are within [0, 3]

    # Debug: Print unique values in the mask
      print(f"Cleaned unique values in mask for {file_id}:", np.unique(mask_data))

    # Stack images into multi-channel format
      image_data = np.stack(image_data, axis=-1)
      mask_data = np.squeeze(mask_data)

      return image_data, mask_data

    def load_nifti(self, file_id, key):
        file_path = f"{ROOT_DIR}/BraTS2021_{file_id}_{key}.nii.gz"
        nii_file = nib.load(file_path)
        return np.asarray(nii_file.get_fdata())

In [77]:
# 3D U-Net Model
def conv_block(input_tensor, num_filters, kernel_size=3):
    x = Conv3D(num_filters, kernel_size, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv3D(num_filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def Unet_3D(input_shape, n_classes, n_filters, n_blocks):
    inputs = Input(shape=input_shape)
    x = inputs
    skips = []

    for _ in range(n_blocks):
        x = conv_block(x, n_filters)
        skips.append(x)
        x = MaxPooling3D(2)(x)
        n_filters *= 2

    x = conv_block(x, n_filters)

    for skip in reversed(skips):
        n_filters //= 2
        x = Conv3DTranspose(n_filters, 3, strides=2, padding='same')(x)
        x = concatenate([x, skip])
        x = conv_block(x, n_filters)

    outputs = Conv3D(n_classes, 1, activation='softmax')(x)
    return Model(inputs, outputs)

In [78]:

# Model Configuration
INPUT_SIZE = 64
ENCODING_BLOCKS = 3
NO_OF_FILTERS = 64
LEARNING_RATE = 1e-4
NUM_EPOCHS = 5

model = Unet_3D((INPUT_SIZE, INPUT_SIZE, INPUT_SIZE, PARAMS['n_channels']),
                PARAMS['n_classes'], NO_OF_FILTERS, ENCODING_BLOCKS)
model.compile(optimizer=Adam(LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 64, 64, 64, 4)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_60 (Conv3D)        │ (None, 64, 64, 64, 64) │          6,976 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_56    │ (None, 64, 64, 64, 64) │            256 │ conv3d_60[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_56             │ (None, 64, 64, 64, 64) │              0 │ batch_normalization_5… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_61 (Conv3D)        │ (None, 64, 64, 64, 64) │        110,656 │ activation_56[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_57    │ (None, 64, 64, 64, 64) │            256 │ conv3d_61[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_57             │ (None, 64, 64, 64, 64) │              0 │ batch_normalization_5… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_12          │ (None, 32, 32, 32, 64) │              0 │ activation_57[0][0]    │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_62 (Conv3D)        │ (None, 32, 32, 32,     │        221,312 │ max_pooling3d_12[0][0] │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_58    │ (None, 32, 32, 32,     │            512 │ conv3d_62[0][0]        │
│ (BatchNormalization)      │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_58             │ (None, 32, 32, 32,     │              0 │ batch_normalization_5… │
│ (Activation)              │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_63 (Conv3D)        │ (None, 32, 32, 32,     │        442,496 │ activation_58[0][0]    │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_59    │ (None, 32, 32, 32,     │            512 │ conv3d_63[0][0]        │
│ (BatchNormalization)      │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_59             │ (None, 32, 32, 32,     │              0 │ batch_normalization_5… │
│ (Activation)         

 Total params: 25,679,044 (97.96 MB)

 Trainable params: 25,673,412 (97.94 MB)

 Non-trainable params: 5,632 (22.00 KB)

In [79]:

# Split file IDs for training and validation
train_ids = file_ids[:1]  # Use one file for training; update this as more files are added
val_ids = file_ids[1:]    # Use additional files for validation

# Create Data Generators
train_gen = DataGenerator(train_ids, **PARAMS)
val_gen = DataGenerator(val_ids, **PARAMS)


In [83]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=NUM_EPOCHS,
    callbacks=[
        ModelCheckpoint(f"{ROOT_DIR}/model_checkpoint.keras", save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
    ]
)


Unique values in mask for 00495: [0.00000000e+00 4.29655471e-16 2.62447884e-14 ... 4.00000000e+00
 4.00000000e+00 4.00000000e+00]
Epoch 1/5
Unique values in mask for 00495: [0.00000000e+00 4.29655471e-16 2.62447884e-14 ... 4.00000000e+00
 4.00000000e+00 4.00000000e+00]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53s/step - accuracy: 0.3590 - loss: 1.5283

ValueError: Must provide at least one structure

In [ ]:

# Visualization
def visualize_prediction(model, generator, index=0):
    X, y_true = generator[index]
    y_pred = model.predict(X)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(np.squeeze(y_true[0, :, :, 32]), cmap='gray')
    plt.title("Ground Truth")
    plt.subplot(1, 2, 2)
    plt.imshow(np.argmax(y_pred[0, :, :, 32, :], axis=-1), cmap='gray')
    plt.title("Prediction")
    plt.show()

visualize_prediction(model, train_gen)

 **BraTS Challenge** focuses on analyzing and segmenting gliomas using multi-parametric MRI (mpMRI) data. The dataset includes:

1. **MRI Modalities**:
   - Native T1-weighted (T1)
   - Post-contrast T1-weighted (T1Gd)
   - T2-weighted (T2)
   - T2 Fluid Attenuated Inversion Recovery (T2-FLAIR)

2. **File Format**:
   - MRI scans are provided as **NIfTI files (.nii.gz)**.
   - Associated de-identified DICOM files (.dcm) might be released later.

3. **Annotations**:
   - Tumor sub-regions are segmented and labeled as:
     - **GD-enhancing tumor (ET)**: Label 4
     - **Peritumoral edematous/invaded tissue (ED)**: Label 2
     - **Necrotic tumor core (NCR)**: Label 1

4. **Data Preprocessing**:
   - Images are **co-registered** to a common anatomical template.
   - Interpolated to a uniform voxel resolution of **1 mm³**.
   - Skull-stripped.

5. **Clinical Background**:
   - The dataset includes **multi-institutional, routine clinically-acquired mpMRI scans**.
   - All scans are pathologically confirmed and have MGMT promoter methylation status.

This dataset is designed for tasks such as tumor segmentation, classification, or related applications in glioma research. It is pre-processed for consistency and quality and includes detailed ground truth annotations validated by expert neuroradiologists.
